# Frehoy rework of new models
* Added C forecast partition

## Imports

In [ ]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import geopandas as gpd


In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel

In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
#from sklearn.utils.fixes import signature
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection

In [ ]:
#import xgboost as xgb
#from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from views.apps.osa import osa
from views.utils import dbutils
from views.utils import pyutils
from views.apps.ensemble import calibrate
from views.apps.evaluate import eval_tex

# Functions

In [ ]:
# The datashifter function returns train or test datasets for the specified periods
# The original df to shift is sent as parameter
# option not to shift a list of specified features

def datashifter_split(df, stepsize, t_split_start, t_split_end, col_outcome, features, nonshifted = []):
#    features = list(set(features))
     # Shift features
    df_step = df[features].groupby(level=1).shift(stepsize)
    # Don't shift outcome and a couple of other columns
    df_step[col_outcome] = df[col_outcome]
#    for var in nonshifted:
    df_step[nonshifted] = df[nonshifted] 
    # Subset times
    df_shifted = df_step.loc[t_split_start:t_split_end]
    # Remove missing observations
    df_shifted = df_shifted.dropna()

    Y_split = np.asarray(df_shifted[col_outcome])
    X_split = np.asarray(df_shifted[features])
    df_id = df_shifted[col_outcome]
    df_id.columns = [col_outcome]
    df_temp = df_shifted
    return (X_split,Y_split,df_id,df_temp)

In [ ]:
# Calibration function

def calibrator(yscore,Y):
    calratio =  Y.mean()/yscore.mean()
    calscore = yscore * calratio
    return(calscore)

def logiter(yscore,zerocorrection):
    yscore[yscore < zerocorrection] =  zerocorrection
    yscore[yscore > 1-zerocorrection] = 1-zerocorrection
    logit = np.log(yscore/(1-yscore))
    return(logit)

def invertlogiter(logitscore):
    invertlogit = np.exp(logitscore)/(1+np.exp(logitscore))
    return(invertlogit)
    

# Importing and setting up data

In [ ]:
from views.utils import config
from views.utils import datautils
import os
dir_scratch = config.CONFIG['dirs']['dir_scratch']
dir_cache = os.path.join(dir_scratch, "cache_model_development")
pyutils.create_dir(dir_cache)

In [ ]:
# Importing and setting up data
timevar = "month_id"
groupvar = "country_id"

if True:
    # Fetch new cm data from flat
    ids = [timevar, groupvar]
    fqtable = "flat.cm_global_1" # One imputation, global cm
    df_flat = dbutils.db_to_df(fqtable=fqtable, ids=ids)

    df_flat.shape
    # Fetch conflict history data from preflight
    ids = [timevar, groupvar]
    fqtable = "preflight.flight_cm"
    df_preflight = dbutils.db_to_df(fqtable=fqtable, ids=ids)
    #df3 = df.dropna()
    df_preflight.shape
    datautils.write_parquet(df_flat, os.path.join(dir_cache, "df_flat.parquet"))
    datautils.write_parquet(df_preflight, os.path.join(dir_cache, "df_preflight.parquet"))

In [ ]:
# Read cache 
df_flat = datautils.load_parquet(os.path.join(dir_cache, "df_flat.parquet"))
df_preflight = datautils.load_parquet(os.path.join(dir_cache, "df_preflight.parquet"))

# Importing feature lists from yaml

In [ ]:
# The yaml file below specifies a number of sets of variables that are useful to define models or to manage data
featlists = pyutils.load_yaml("../../projects/model_development/featlists_hh.yaml")

# Feature lists used to handle data/dependent variables and lags
colsfrompreflight = featlists['colsfrompreflight']
currentconflict = featlists['currentconflict']
# Features for filling in missing
feat_acled_v_lags = featlists['feat_acled_v_lags']
feat_acled_v_months = featlists['feat_acled_v_months']
feat_ged_lags = featlists['feat_ged_lags']
feat_ged_months = featlists['feat_ged_months']
# WDI features broken up in categories:
feat_wdi_rest = featlists['feat_wdi_rest']
feat_wdi_milexp = featlists['feat_wdi_milexp']
feat_wdi_unemp = featlists['feat_wdi_unemp']
feat_wdi_macro = featlists['feat_wdi_macro']
feat_wdi_cpia = featlists['feat_wdi_cpia']
feat_wdi_fdi = featlists['feat_wdi_fdi']
feat_wdi_gender = featlists['feat_wdi_gender']
feat_wdi_poverty = featlists['feat_wdi_poverty']
feat_wdi_ineq = featlists['feat_wdi_ineq']
feat_wdi_refugees = featlists['feat_wdi_refugees']
feat_wdi_gdp = featlists['feat_wdi_gdp']
feat_wdi_agri = featlists['feat_wdi_agri']
feat_wdi_debt = featlists['feat_wdi_debt']
feat_wdi_mort = featlists['feat_wdi_mort']
feat_wdi_educ = featlists['feat_wdi_educ']
feat_wdi_health = featlists['feat_wdi_health']
feat_wdi_demog = featlists['feat_wdi_demog']
feat_wdi_trade = featlists['feat_wdi_trade']
feat_wdi_pkos = featlists['feat_wdi_pkos']
# Conflict history features
feat_neibhist = featlists['feat_neibhist']
feat_cfshort = featlists['feat_cfshort']
feat_cflag = featlists['feat_cflag']
feat_acled_pr = featlists['feat_acled_pr']

# Vdem features
feat_vdem = featlists['feat_vdem']
feat_vdem_hlc = featlists['feat_vdem_hlc']
# Misc features
feat_cnt = featlists['feat_cnt']
feat_inst = featlists['feat_inst']
demog = featlists['demog']
econ = featlists['econ']
feat_reign = featlists['feat_reign']
feat_couprisk = featlists['feat_couprisk']
feat_icgcw = featlists['feat_icgcw']
#feat_x = featlists['']

In [ ]:
# Merge the two data sources flat and preflight together in a global df

cols_flat = list(df_flat.columns)
cols_flat.remove('ged_best_sb')
cols_flat.remove('ged_best_ns')
cols_flat.remove('ged_best_os')

#colswanted = [col for col in cols2 if not col in cols3]
df_both =  df_preflight[colsfrompreflight].merge(df_flat[cols_flat], how='outer', left_index=True, right_index=True)

df = df_both.loc[1:550]
df.shape, df_both.shape

# Lagging some features

In [ ]:
# Data from vdem
feat_vdem_orig = feat_vdem.copy()

In [ ]:
# Merging WDI features broken up in categories into a large feature list
feat_wdi_all = feat_wdi_rest + feat_wdi_gdp + feat_wdi_agri + feat_wdi_debt + feat_wdi_mort + feat_wdi_educ
feat_wdi_all = feat_wdi_all + feat_wdi_health + feat_wdi_demog + feat_wdi_trade + feat_wdi_pkos
feat_wdi_all = feat_wdi_all + feat_wdi_refugees + feat_wdi_cpia + feat_wdi_gender + feat_wdi_ineq + feat_wdi_poverty
feat_wdi_all = feat_wdi_all + feat_wdi_fdi + feat_wdi_macro + feat_wdi_gender + feat_wdi_unemp
feat_wdi_all = feat_wdi_all + feat_wdi_milexp

feat_wdi_all = pyutils.dedup_list(feat_wdi_all)

In [ ]:
def add_lagged_features(df, featset, lag):
    prefix = "tlag_" + str(lag) + "_"
    df_shifted = df[featset].groupby(level=1).shift(lag)
    df_shifted = df_shifted.add_prefix(prefix)
    cols = list(df_shifted.columns)
    newcols = featset + cols
    df = df.join(df_shifted)
    return(df,newcols)

df, feat_vdem = add_lagged_features(df,feat_vdem,12)
#df, feat_vdem_hlc = add_lagged_features(df,feat_vdem_hlc,12)
df, feat_wdi_pkos = add_lagged_features(df,feat_wdi_pkos,12)
df, feat_wdi_trade = add_lagged_features(df,feat_wdi_trade,12)
df, feat_wdi_debt = add_lagged_features(df,feat_wdi_debt,12)
df, feat_wdi_milexp = add_lagged_features(df,feat_wdi_milexp,12)
df, feat_wdi_cpia = add_lagged_features(df,feat_wdi_cpia,12)
df, feat_wdi_fdi = add_lagged_features(df,feat_wdi_fdi,12)
df, feat_wdi_refugees = add_lagged_features(df,feat_wdi_refugees,12)
  

In [ ]:
feat_vdem_12 = feat_vdem.copy()
df, feat_vdem = add_lagged_features(df,feat_vdem,60)

In [ ]:
len(feat_vdem), len(feat_vdem_12), len(feat_vdem_orig)

# Filling in zeros for missing for some features

In [ ]:
# Feature lists retrived from yaml file above
feat_acled_v = feat_acled_v_lags + feat_acled_v_months 

df[feat_acled_v_lags] = df[feat_acled_v_lags].fillna(0)
df[feat_acled_v_months] = df[feat_acled_v_months].fillna(999)
df[feat_ged_lags] = df[feat_ged_lags].fillna(0)
df[feat_ged_months] = df[feat_ged_months].fillna(999)

# Copy the df to an Africa-only df

In [ ]:
# Africa-only df
df_africa = df[df['in_africa']==1].copy()
df_glob = df.copy()
df_africa.shape

# Specify remaining feature sets

In [ ]:

feat_cflong = colsfrompreflight.copy()
for var in currentconflict:
#    print(var)
    feat_cflong.remove(var)

feat_vdem_hlc_africa = feat_vdem_hlc + ['in_africa']

feat_demog = feat_wdi_demog + demog

feat_econ = feat_wdi_gdp + feat_wdi_trade + econ

feat_reign_africa = feat_reign + ['in_africa']

In [ ]:
feat_all = feat_cflong + feat_acled_v
feat_all = feat_all + feat_acled_pr
feat_all = feat_all + feat_inst
feat_all = feat_all + feat_demog
#feat_all = feat_all + feat_econ
feat_all = feat_all + feat_vdem_hlc
feat_all = feat_all + feat_cnt
feat_all = feat_all + feat_wdi_all
feat_all = feat_all + feat_reign
feat_all = feat_all + feat_vdem
feat_all

feat_all = pyutils.dedup_list(feat_all)

In [ ]:
# Specifying models

treenos = 500

feat_in_africa = ['in_africa']
models = []

main_outcome_sb = 'ged_tx_sb_25'
main_outcome_os = 'ged_tx_os_25'
main_outcome_ns = 'ged_tx_ns_25'

m_vdem_sb = {
    'name': "vdem_sb",
    'features': feat_vdem + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=10, n_jobs=-1),
    'outcome': main_outcome_sb
}

m_wdi_all_sb = {
    'name': "wdi_all_sb",
    'features': feat_wdi_all + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=20, n_jobs=-1),
    'outcome': main_outcome_sb
}
m_reign_sb = {
    'name': "reign_sb",
    'features': feat_reign + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=5, n_jobs=-1),
    'outcome': main_outcome_sb
}

m_all_sb = {
    'name': "all_sb",
    'features': feat_all,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=20, n_jobs=-1),
    'outcome': main_outcome_sb
}
m_vdem_os = {
    'name': "vdem_os",
    'features': feat_vdem + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=10, n_jobs=-1),
    'outcome': main_outcome_os
}

m_wdi_all_os = {
    'name': "wdi_all_os",
    'features': feat_wdi_all + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=20, n_jobs=-1),
    'outcome': main_outcome_os
}
m_reign_os = {
    'name': "reign_os",
    'features': feat_reign + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=5, n_jobs=-1),
    'outcome': main_outcome_os
}

m_all_os= {
    'name': "all_os",
    'features': feat_all,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=20, n_jobs=-1),
    'outcome': main_outcome_os
}
m_vdem_ns = {
    'name': "vdem_ns",
    'features': feat_vdem + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=10, n_jobs=-1),
    'outcome': main_outcome_ns
}

m_wdi_all_ns = {
    'name': "wdi_all_ns",
    'features': feat_wdi_all + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=20, n_jobs=-1),
    'outcome': main_outcome_ns
}
m_reign_ns = {
    'name': "reign_ns",
    'features': feat_reign + feat_in_africa,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=5, n_jobs=-1),
    'outcome': main_outcome_ns
}

m_all_ns= {
    'name': "all_ns",
    'features': feat_all,
    'estimator': RandomForestClassifier(n_estimators=treenos, max_features=20, n_jobs=-1),
    'outcome': main_outcome_ns
}

models.append(m_vdem_sb)
models.append(m_reign_sb)
models.append(m_wdi_all_sb)
models.append(m_all_sb)
models.append(m_vdem_os)
models.append(m_reign_os)
models.append(m_wdi_all_os)
models.append(m_all_os)
models.append(m_vdem_ns)
models.append(m_reign_ns)
models.append(m_wdi_all_ns)
models.append(m_all_ns)


In [ ]:
periods = []
period1 = {
    'pno': 0,
    'period': "calib",
    'train_start': 121,
    'train_end': 396,
    'test_start': 397,
    'test_end': 432,
#    'models': models
}
period2 = {
    'pno': 1,
    'period': "test",
    'train_start': 121,
    'train_end': 432,
    'test_start': 433,
    'test_end': 468
#    'models': models
}
period3 = {
    'pno': 3,
    'period': "fcast",
    'train_start': 121,
    'train_end': 480,
    'test_start': 481,
    'test_end': 518
}
periods.append(period1)
periods.append(period2)
periods.append(period3)

In [ ]:
#[1,2,3,4,5,6,9,12,15,18,21,24,27,30,33,36]
allsteps = list(range(1,36+1))
steps = []
for stepsize in [1,6,12,24,36]:
    step = {
        'stepsize': stepsize,
#        'periods': periods
    }
    steps.append(step)
    

In [ ]:
%%time
# New: only training 
# Holds all the results in nested dictionaries
# results_glob trained on global data, and results_afr on Africa only
import copy
#df_glob = df.copy()
results = dict()

#for trainingregion in ['global','africa']:
for trainingregion in ['global']:
    print(trainingregion)
    for step in steps:
        # Create a key in results dict for this step containing another dict
        results[step['stepsize']] = dict()
        print("Step: ", step['stepsize'])
        for period in periods:
            print(period['period'], 'partition')
            # Create a key in results for this period in this step
            results[step['stepsize']][period['period']] = dict()
            for model in models:
                print( model['outcome'])
    
                if trainingregion == 'global':
                    model['X_train'],model['Y_train'],model['train_IDs'],model['df'] = datashifter_split(df_glob, step['stepsize'],period['train_start'],period['train_end'],model['outcome'],model['features'])
                if trainingregion == 'africa':
                    model['X_train'],model['Y_train'],model['train_IDs'],model['df'] = datashifter_split(df_africa, step['stepsize'],period['train_start'],period['train_end'],model['outcome'],model['features'])
                model_object = copy.copy(model['estimator'])
                model_object.fit(model['X_train'], model['Y_train'])
    #            modelfit = model_object.fit(model['X_train'], model['Y_train'])
                model['fit'] = model_object
                if isinstance(model['estimator'], sklearn.ensemble.RandomForestRegressor):
                    model['feature_importances'] = model['fit'].feature_importances_
                if isinstance(model['estimator'], sklearn.ensemble.RandomForestClassifier):
                    model['feature_importances'] = model['fit'].feature_importances_
                if isinstance(model['estimator'], sklearn.linear_model.LogisticRegression):
                    model['coefs'] = model['fit'].coef_
                msg = "%s: %02d %s: %s. %s: %s  %s: %s  " % ('Step', step['stepsize'], 'Period', period['period'], 
                                                     'Done model', model['name'], 'Observations for training', len(model['X_train']))
                print(msg)
                results[step['stepsize']][period['period']][model['name']] = copy.deepcopy(model)
    if trainingregion == 'global':
        results_glob = results.copy()
    if trainingregion == 'africa':
        results_afr = results.copy()

print("Training done")

In [ ]:
%%time
# Predict probabilities from trained models; using datashifter_split; global and Africa
# Outcome for testing:

#col_outcome = "ged_tx_sb_25"
fat_tr = 25

# globally trained models
print("Models trained on global data")
results = results_glob
df = df_glob
for step in steps:
    s = step['stepsize']
    print(s)
    for period in periods:
        p = period['period']
        print(p)
        for model in models:
            m = model['name']
            col_outcome = model['outcome']
            for r in ['africa']:
                results[s][p][m][r] = dict()
                if r == 'global':
                    (results[s][p][m][r]['X_test'],results[s][p][m][r]['Y_test'],results[s][p][m][r]['test_ids'],results[s][p][m][r]['df']) = datashifter_split(df_glob,step['stepsize'],period['test_start'],period['test_end'],col_outcome,model['features'])
                if r == 'africa':
                    (results[s][p][m][r]['X_test'],results[s][p][m][r]['Y_test'],results[s][p][m][r]['test_ids'],results[s][p][m][r]['df']) = datashifter_split(df_africa.fillna(0),step['stepsize'],period['test_start'],period['test_end'],col_outcome,model['features'])
                results[s][p][m][r]['test_ids'].columns = col_outcome
                results[s][p][m][r]['test_outcome'] = col_outcome
#                print('region: ',r)
                if (isinstance(model['estimator'], sklearn.ensemble.RandomForestClassifier) or
                    isinstance(model['estimator'], sklearn.linear_model.LogisticRegression) or 
                    isinstance(model['estimator'], sklearn.ensemble.AdaBoostClassifier) or 
                    isinstance(model['estimator'], sklearn.ensemble.GradientBoostingClassifier)):
                    results[s][p][m][r]['pred'] =  results[s][p][m]['fit'].predict(results[s][p][m][r]['X_test'])
                    results[s][p][m][r]['yscore'] =  results[s][p][m]['fit'].predict_proba(results[s][p][m][r]['X_test'])[:,1]
                if (isinstance(model['estimator'], sklearn.ensemble.RandomForestRegressor) or
                   isinstance(model['estimator'], sklearn.linear_model.Lasso)):
#                    results[s][p][m][r]['yscore'] =  results[s][p][m][r]['fit'].predict(results[s][p][m][r]['X_test'])
                    results[s][p][m][r]['yscore'] =  FatalitiesToProbabilities(results[s][p][m]['fit'].predict(results[s][p][m][r]['X_test']),fat_tr,1)
                    results[s][p][m][r]['pred'] =  FatalitiesToPred(results[s][p][m]['fit'].predict(results[s][p][m][r]['X_test']),fat_tr)
                # Recode 0 and 1 
                results[s][p][m][r]['yscore'][results[s][p][m][r]['yscore'] < 0.0001] = 0.0001
                results[s][p][m][r]['yscore'][results[s][p][m][r]['yscore'] > 0.9999] = 0.9999
                results[s][p][m][r]['ysc_df'] = pd.DataFrame(results[s][p][m][r]['yscore'], index = results[s][p][m][r]['test_ids'].index)
                results[s][p][m][r]['ysc_df'].columns = [m]
                results[s][p][m][r]['crosstab'] = pd.crosstab(results[s][p][m][r]['pred'],results[s][p][m][r]['Y_test'],rownames=['Predicted'], colnames=['Actual'],margins=True)
      #          results[s][p][m][r]['accuracy'] = accuracy_score(results[s][p][m][r]['Y_test'],results[s][p][m][r]['pred'])
                results[s][p][m][r]['accuracy'] = accuracy_score(results[s][p][m][r]['Y_test'],results[s][p][m][r]['pred'])
                results[s][p][m][r]['AUPR'] = average_precision_score(results[s][p][m][r]['Y_test'], results[s][p][m][r]['yscore'])
                results[s][p][m][r]['AUROC'] = roc_auc_score(results[s][p][m][r]['Y_test'], results[s][p][m][r]['yscore'])
                results[s][p][m][r]['shape'] = results[s][p][m][r]['yscore'].shape
    #            results[s][p][m]['featzip'] = zip(results[s][p][m]['features'], results[s][p][m]['fit'].feature_importances_)

                msg = "%s= %02d %s %s %s %s: %f %s: %f %s: %f " % ('s', s,  p, 
                                                     results[s][p][m]['name'], r,  'Accuracy', results[s][p][m][r]['accuracy'], 
                                                      'AUPR', results[s][p][m][r]['AUPR'], 'AUROC', results[s][p][m][r]['AUROC'])
                print(msg)
results_glob = results

In [ ]:
# Collect predictions in a df
df_cm_hh_a_predict = df_africa[['ged_tx_sb_25','ged_tx_os_25','ged_tx_ns_25']].loc[397:432].copy()
df_cm_hh_b_predict = df_africa[['ged_tx_sb_25','ged_tx_os_25','ged_tx_ns_25']].loc[433:468].copy()
df_cm_hh_c_predict = df_africa[['ged_tx_sb_25','ged_tx_os_25','ged_tx_ns_25']].loc[477:514].copy()


for step in steps:
    s = step['stepsize']
    print(s)
    for period in periods:
        p = period['period']
        print(p)
        for model in models:
            m = model['name']
            print(m)
            colname = 'ss.' + m + '.' + str(s)
            for r in ['africa']:
                if p == 'calib':
                    df_cm_hh_a_predict[colname] = results[s][p][m][r]['ysc_df']
                if p == 'test':
                    df_cm_hh_b_predict[colname] = results[s][p][m][r]['ysc_df']
                if p == 'fcast':
                    df_cm_hh_c_predict[colname] = results[s][p][m][r]['ysc_df']
                

In [ ]:
#dbutils.df_to_db(df=df_cm_hh_a_predict,fqtable='newpipe.cm_hh_a_predict_fh')
#dbutils.df_to_db(df=df_cm_hh_b_predict,fqtable='newpipe.cm_hh_b_predict_fh')
#dbutils.df_to_db(df=df_cm_hh_c_predict,fqtable='newpipe.cm_hh_c_predict_fh') 

In [ ]:
# Get featuer importances
r = "africa"
featimps = []
for step in steps:
    s = step["stepsize"]
    p = "fcast"
    for model in models:
        m = model["name"]
        fi_dict = dict()
        features = model["features"]
        print(s, p, m)
        fi = results[s][p][m]["fit"].feature_importances_
        for value, feature in zip(fi, features):
            fi_dict[feature] = value
        fi_result = {"name": f"{m}.{s}", "feature_importances": fi_dict} 
        featimps.append(fi_result)

In [ ]:
# Persist feature importances
import json
def persist_featimps(featimps, fqtable):
    q_create = f"DROP TABLE IF EXISTS {fqtable}; CREATE TABLE {fqtable} (featimp JSON);"
    dbutils.execute_query(q_create)
    for featimp in featimps:
        q_insert = f"INSERT INTO {fqtable} (featimp) VALUES (:featimp)"
        params = {"featimp": json.dumps(featimp)}
        dbutils.execute_query(q_insert, **params)
persist_featimps(featimps, fqtable="newpipe.featimps_cm_c_global")